In [33]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
from replay import Replay, State, Team
from bots.lgbm.preprocessing    import remove_menu_and_pause
from bots.lgbm.preprocessing    import who_scored
from bots.lgbm.preprocessing    import game_state_to_numpy

from typing import Optional
from sklearn.metrics import confusion_matrix, classification_report
from joblib import dump, load

import matplotlib.pyplot as plt
import numpy as np

In [36]:

def boolarr_to_class(y : [bool]) -> int:
    def boolarr_to_dirstate(y : [bool]) -> np.array:
        result = [0,0,0]
        if y[0]: #UP key pressed
            result[0] += 1
        if y[1]: #down key pressed
            result[0] -= 1

        if y[2]: #left
            result[1] += 1
        if y[3]: #right
            result[1] -= 1

        if y[4]:
            result[2] += 1

        return np.array(result)
    
    y = tuple(boolarr_to_dirstate(y))
    
    if y == tuple([0,0,0]):
        return 0
    elif y == tuple([0,0,1]):
        return 1
    
    elif y == tuple([-1,0,0]):
        return 2
    elif y == tuple([-1,0,1]):
        return 3
    
    elif y == tuple([0,-1,0]):
        return 4
    elif y == tuple([0,-1,1]):
        return 5
    
    elif y == tuple([1,0,0]):
        return 6
    elif y == tuple([1,0,1]):
        return 7
    
    elif y == tuple([0,1,0]):
        return 8
    elif y == tuple([0,1,1]):
        return 9
    
    elif y == tuple([1,1,0]):
        return 10
    elif y == tuple([1,1,1]):
        return 11
    
    elif y == tuple([-1,-1,0]):
        return 12
    elif y == tuple([-1,-1,1]):
        return 13
    
    elif y == tuple([-1,1,0]):
        return 14
    elif y == tuple([-1,1,1]):
        return 15
    
    elif y == tuple([1,-1,0]):
        return 16
    elif y == tuple([1,-1,1]):
        return 17
def prepare_train_data(episodes, stack_frames=4):
    # Take resultative episodes as training data
    train_x, train_y = [], []
    for episode in episodes:
        scorer = who_scored(episode)
        if scorer is not None:
            if scorer == Team.Red:
                continue
                transform_blue_pivotal(episode)
            
            x = []
            y = []
            for e in episode:
                e_x, e_y = game_state_to_numpy(e, scorer)
                if e_x is not None and e_y is not None:
                    x.append(e_x)
                    y.append(e_y)
            
            # If there was no target player - ignore
            if x is not None and y is not None:
                train_x.append(x)
                train_y.append(y)
                
    # Stack n-frames
    x = []
    y = []
    for episode_i in range(len(train_x)):
        episode_x      = train_x[episode_i]
        episode_y      = train_y[episode_i]
        
        # Stacked samples
        train_sample = [episode_x[0] for _ in range(stack_frames)]
        
        x.append(np.concatenate(train_sample).ravel())
        y.append(boolarr_to_class(np.array(episode_y[0])))
        
        for frame_i in range(1, len(episode_x)):
            frame_x = episode_x[frame_i]
            frame_y = episode_y[frame_i]
            
            # Move stack
            for i in range(1, stack_frames):
                train_sample[i] = train_sample[i - 1]
            train_sample[0] = frame_x

            x.append(np.concatenate(train_sample).ravel())
            y.append(boolarr_to_class(np.array(frame_y)))
            
    return x, y

In [37]:
def transform_blue_pivotal(frames) -> Replay:
    """
    Inplace transformation of disk positions and velocities based on the pivotal team side.
    Inverse x-axis basically.
    
    frames: a list of replay frames
    """
    for frame in frames:
        if frame.players == None:
            continue
            
        for player in frame.players:
            # Inverse position and velocity along x-axis
            player.disc.x  = -player.disc.x
            player.disc.vx = -player.disc.vx
            
            # Swap left and right actions
            temp            = player.input[2]
            player.input[2] = player.input[3]
            player.input[3] = temp
            
        # Inverse position and velocity of the ball along x-axis
        frame.ball.x  = -frame.ball.x
        frame.ball.vx = -frame.ball.vx

In [38]:
import os

def prepare_train_data_folder(path):
    all_episodes = []
    num_games    = 0
    for file in os.listdir(path):
        if file[-4:] != ".bin":
            continue
            
        num_games += 1
        
        # Parse replay and etract game epsiodes
        _, replay = Replay(open(os.path.join(path, file), mode="rb").read())
        episodes = remove_menu_and_pause(replay)
        all_episodes.extend(episodes)
        
    print("Num episodes: {}".format(len(all_episodes)))
    print("Num games: {}".format(num_games))
    return prepare_train_data(all_episodes, 1)

In [39]:
import json

with open("../preprocessed/statistics.json") as f:
    names = json.load(f)

In [40]:
max_replays = 0
max_name    = None
for name in names.keys():
    if len(names[name]) > max_replays:
        max_replays = len(names[name])
        max_name = name

In [41]:
max_replays

26

In [42]:
max_name

'Saviour'

In [43]:
all_episodes = []
for relative_path in names[max_name]:
    path = ".."
    # Parse replay and etract game epsiodes
    _, replay = Replay(open(os.path.join(path, relative_path), mode="rb").read())
    episodes = remove_menu_and_pause(replay)
    all_episodes.extend(episodes)

In [44]:
len(all_episodes)

348

In [45]:
t1, t2 = prepare_train_data(all_episodes, 4)#prepare_train_data_folder("replays/cups/edition7/")
del all_episodes

In [46]:
import lightgbm as lgb

In [ ]:
imitator = lgb.LGBMClassifier(n_estimators=300)

In [ ]:
imitator.fit(t1, t2)

In [ ]:
t2_pred = imitator.predict(t1)

In [ ]:
from sklearn.utils.multiclass import unique_labels

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
plot_confusion_matrix(t2, t2_pred.astype(int), list(range(18)))
plt.show()

In [ ]:
# 100 estimators
print(classification_report(t2, t2_pred))

In [ ]:
dump(imitator, 'imitator1.joblib')

In [ ]:
plot_confusion_matrix(t2, t2_pred.astype(int), list(range(18)))
plt.show()

In [ ]:
# 200 estimators
print(classification_report(t2, t2_pred))

#### 400 estimators
print(classification_report(t2, t2_pred))